In [1]:
import pandas as pd
import json
import sklearn.svm as svm
from sklearn.linear_model import Lasso, LassoCV
from sklearn.feature_selection import SelectFromModel
import numpy as np
from sklearn.neural_network import MLPClassifier
import script as sc
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [2]:
## Change the name to get your path
user = 'Yash'

## Build test and train Datasets

In [3]:
data = pd.DataFrame(sc.get_complete(user))

In [4]:
copy = data[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df']].copy(deep=True)
copy.loc[-1] = copy.loc[0]  # adding a row
copy.index = copy.index + 1  # shifting index
copy.sort_index(inplace=True)
copy.columns = 'last.' + copy.columns
data = pd.concat([data, copy], axis = 1)
data = data.drop(data.index[len(data)-1])

In [5]:
trainX, trainY, testX, testY = sc.get_test_train(data, 0)

In [ ]:
data.shape

## SVM

In [305]:
clf = svm.SVC(cache_size=7000)
clf.fit(trainX, trainY)

SVC(C=1.0, cache_size=7000, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [306]:
clf.score(testX,testY)

0.7878451505480782

## Neural Network

In [ ]:
n_net = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(2000, 1000), random_state=1, verbose = True)
n_net.fit(trainX,trainY)

In [9]:
np.sum(n_net.predict(testX) == testY) / len(testX)

0.8333739766591186

In [12]:
predictedVals = n_net.predict(testX)
print (confusion_matrix(testY,predictedVals,labels = ['sedentary','non-sedentary']))
f1_score(testY == 'sedentary', predictedVals == 'sedentary',labels = ['sedentary','non-sedentary'])

[[ 9860  2856]
 [ 1927 14062]]


0.8047994123168593

In [13]:
train.shape

(86112, 20)